### DTSA 5511 Cancer Detection Kaggle Mini-Project

**Step 1**

**Brief Description of the problem and data(5 pts)**

Metastatic Cancer (source : https://www.cancer.gov/types/metastatic-cancer#what)

The process by which cancer cells spread to other parts of the body is called metastatsis. It is also called Stage IV Cancer.
When observed under a microscope and tested in other ways, metastatic cancer cells have features like that of the primary cancer and not like the cells in the place where the metastatic cancer is found. This is how doctors can tell that it is cancer that has spread from another part of the body.

The goal is to create a model to identify metastatic cancer in images taken from larger digital pathology scanes. The images are taken from modified version of PatchCamelyon(PCam) benchmark dataset. 

We are provided with small pathology images. The files are named with an image id. Positive lables indicates that the center 32x32px region of a patch contains atleast one pixel of tumor tissue. Tumor tissue in the outer region of the patch does not influence the label. This outer region is provided to enable fully-convolutional models that do not use zero-padding, to ensure consistent behavior when applied to a whole-slide image.


In [ ]:
# import necessary libraries


**Step 2**

**Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data (15 pts)**

Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis? 


**Step 3**

**DModel Architecture (25 pts)**

Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem. Compare multiple architectures and tune hyperparameters.

**Step 4**

**Results and Analysis (35 pts)** 

Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.

Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.

**Step 5**

**Conclusion (15 pts)**

Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?

**Step 6**

**Produce Deliverables: High-Quality, Organized Jupyter Notebook Report, GitHub Repository, and screenshot of Kaggle leaderboard (35 points)**

